<h3>Model</h3>

In [9]:
# python standard libraries
import os
import random
import fnmatch
import datetime
import pickle
import re

# data processing
import numpy as np
np.set_printoptions(formatter={'float_kind':lambda x: "%.4f" % x})

import pandas as pd
pd.set_option('display.width', 300)
pd.set_option('display.float_format', '{:,.4f}'.format)
pd.set_option('display.max_colwidth', 200)

# tensorflow
import tensorflow as tf
import keras
from keras.models import Sequential  # V2 is tensorflow.keras.xxxx, V1 is keras.xxx
from keras.layers import Conv2D, MaxPool2D, Dropout, Flatten, Dense
from keras.models import load_model
from keras.utils import np_utils

# sklearn
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

# imaging
import cv2
from imgaug import augmenters as img_aug
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

In [10]:
def mapDirection(dir):
    if(int(dir) == -1):
        return 0
    if(int(dir) == 1):
        return 90
    if(int(dir) == 0):
        return 180
        


In [11]:
data_dir = './Data'
data_folders = os.listdir(data_dir)
file_list = []

for dir in data_folders:
    file_list = file_list + list(map(lambda d: dir+ "/" + d,os.listdir(os.path.join(data_dir,dir))))
image_paths = []
steering_direction = []
pattern = "*.png"
for filename in file_list:
    if fnmatch.fnmatch(filename, pattern):
        image_paths.append(os.path.join(data_dir,filename))
        direction = re.sub('.png','',re.sub('.*dir_','',filename))  # 092 part of video01_143_092.png is the angle. 90 is go straight
        steering_direction.append(mapDirection(direction))

The size of the validation set is not a good split. But we want to train the model on as much data as possible.

In [12]:
X_train, X_valid, y_train, y_valid = train_test_split( image_paths, steering_direction, test_size=0.1)
print("Training data: %d\nValidation data: %d" % (len(X_train), len(X_valid)))

Training data: 435
Validation data: 49


In [13]:
def zoom(image):
    zoom = img_aug.Affine(scale=(1.1, 1.3))  # zoom from 100% (no zoom) to 130%
    image = zoom.augment_image(image)
    return image

Random augmentation of the images provides a more robust data set.
Because of a miss alignment of the wheels of the car we can not flip these images.

In [15]:
def random_augment(image, steering_angle):
    if np.random.rand() < 0.5:
        image = zoom(image)
    return image, steering_angle

Image processing is necesary to use the images with the nvidia model.

Documentation about the nvidia model says that it is better to convert the image from RGB to YUV.
This line is comented because of an issue when converting live images to YUV while the car is autonomous driving.

In [16]:
def img_preprocess(image):
    height, _, _ = image.shape
    image = image[int(height/2):,:,:]  # remove top half of the image, as it is not relevant for lane following
    #image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)  # Nvidia model said it is best to use YUV color space
    image = cv2.GaussianBlur(image, (3,3), 0)
    image = cv2.resize(image, (200,66)) # input image size (200,66) Nvidia model
    image = image / 255 # normalizing the values of the image. We dont need to normalize it in the model anny more
    return image

Research teached us that the nvidea model is an efficiënt model to use for autonomous driving.

It uses elu activation functions, this is usefull for training the model (this is becaus unlike the relu it backpropagates negative values).

In [17]:
def nvidia_model():
    model = Sequential(name='Nvidia_Model')
    
    # elu=Expenential Linear Unit, similar to leaky Relu
    # skipping 1st hiddel layer (nomralization layer), as we have normalized the data
    
    # Convolution Layers
    model.add(Conv2D(24, (5, 5), strides=(2, 2), input_shape=(66, 200, 3), activation='elu')) 
    model.add(Conv2D(36, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(48, (5, 5), strides=(2, 2), activation='elu')) 
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Conv2D(64, (3, 3), activation='elu')) 
    
    # Fully Connected Layers
    model.add(Flatten())
    model.add(Dropout(0.2)) # not in original model. added for more robustness
    model.add(Dense(100, activation='elu'))
    model.add(Dense(50, activation='elu'))
    model.add(Dense(10, activation='elu'))
    
    # output layer: turn angle (from 45-135, 90 is straight, <90 turn left, >90 turn right)
    model.add(Dense(activation="sigmoid", units=1))
    
    # since this is a regression problem not classification problem
    model.compile(optimizer = 'adam', loss = 'categorical_hinge', metrics = ['accuracy'])
    
    return model

model = nvidia_model()
print(model.summary())

Model: "Nvidia_Model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 31, 98, 24)        1824      
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 14, 47, 36)        21636     
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 5, 22, 48)         43248     
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 3, 20, 64)         27712     
_________________________________________________________________
dropout_2 (Dropout)          (None, 3, 20, 64)         0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 1, 18, 64)         36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 1152)             

In [18]:
def image_data_generator(image_paths, steering_angles, batch_size, is_training):
    while True:
        batch_images = []
        batch_steering_angles = []
        
        for i in range(batch_size):
            random_index = random.randint(0, len(image_paths) - 1)
            image_path = image_paths[random_index]
            image = cv2.imread(image_paths[random_index])
            steering_angle = steering_angles[random_index]
            if is_training:
                # training: augment image
                image, steering_angle = random_augment(image, steering_angle)
              
            image = img_preprocess(image)
            batch_images.append(image)
            batch_steering_angles.append(steering_angle)
            
        yield( np.asarray(batch_images), np.asarray(batch_steering_angles))

In [19]:
model_output_dir = "./Model"


# saves the model weights after each epoch if the validation loss decreased
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=os.path.join(model_output_dir,'lane_navigation_check.h5'), verbose=1, save_best_only=True)

history = model.fit_generator(image_data_generator( X_train, y_train, batch_size=100, is_training=True),
                              steps_per_epoch=300,
                              epochs=1,
                              validation_data = image_data_generator( X_valid, y_valid, batch_size=100, is_training=False),
                              validation_steps=200,
                              verbose=1,
                              shuffle=1,
                              callbacks=[checkpoint_callback])
# always save model output as soon as model finishes training
model.save(os.path.join(model_output_dir,'lane_navigation_final.h5'))

C:\Users\32492\AppData\Local\Programs\Python\Python39\lib\site-packages\keras\engine\training.py:1972: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


  4/300 [..............................] - ETA: 7:47 - loss: 0.5064 - accuracy: 0.3275

KeyboardInterrupt: 

In [25]:

model = load_model('./Model/lane_navigation_check.h5')
        
def compute_steering_angle( frame):
    preprocessed = img_preprocess(frame)
    X = np.asarray([preprocessed])
    steering_angle = model.predict(X)[0]
    return steering_angle
image = cv2.imread('./Data/capture3/nr3_15_dir_-1.png')
print(compute_steering_angle(image))

[81.9020]


In [28]:
import tensorflow as tf

# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(load_model('./Model/lane_navigation_check.h5'))
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\32492\AppData\Local\Temp\tmpnozcpe4r\assets


INFO:tensorflow:Assets written to: C:\Users\32492\AppData\Local\Temp\tmpnozcpe4r\assets
